# German Lawyer 

A Jupyter notebook to help navigate the residency law in Germany. This project uses local embeddings and models to do RAG (Retreival Augmented Generation) over the German residency law. This means that the model is living locally on the computer, the embeddings are done locally, and the querying is done locally.

You can ask questions like:

* What are the requirements for a Blue Card?
* What are the requirements for a student visa?
* What are the requirements for a work visa?

I've taken the Aufenthaltsgesetz and Aufenthaltsverordnung from Gesetze im Internet as XML and using the Unstructured XML loader, I've loaded them in as a LangChain document.

## Project Steps

1. Load the XML files into a LangChain document
2. Split the document into sections
3. Embeddings
4. Vector Store
5. LLM Setup (Prompt Template & Querying)

### 1: Use LangChain Unstructured XML Loader to Load in the German Residence Law

In [72]:
from langchain.document_loaders import UnstructuredXMLLoader, TextLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

from langchain.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant

from langchain.llms import Ollama

from langchain.chains import RetrievalQA   
from langchain.prompts import PromptTemplate 

import time

from langchain.chains import ConversationalRetrievalChain


### 2: Load & Split the Text

In [43]:
# German Residence Law
# source: https://www.gesetze-im-internet.de/aufenthv/BJNR294510004.html
file = "german-law/laws/Aufenthaltsverordnung/BJNR294510004.xml"

# load German Residence Law XML file with UnstructuredXMLLoader
loader = UnstructuredXMLLoader(file_path = file)
docs = loader.load()

**Recursive Character Text Splitter**

Use recursive character text splitter to split texts into chunks of 1000

In [46]:
# Try with the RecursiveCharacterTextSplitter

r_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
r_texts = r_text_splitter.create_documents([docs[0].page_content])


In [45]:
# Try with the CharacterTextSplitter

c_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
c_texts = c_text_splitter.create_documents([docs[0].page_content])


Created a chunk of size 7988, which is longer than the specified 1500
Created a chunk of size 3185, which is longer than the specified 1500


### 3: Create Vectorstore

In [54]:
# OpenAI Embeddings, Chroma as vectorstore
openai_vectorstore = Chroma.from_documents(documents = r_texts, embedding=OpenAIEmbeddings())
retreiver = openai_vectorstore.as_retriever()

In [30]:
# Ollama Embeddings (openhermes2.5), Qdrant as vectorstore 
# Note: (Chroma does not work, as Ollama creates 4096-dimensional vectors and Chroma accepts 1536-dimensional vectors only)

loader = TextLoader("/Users/ingrid/Developer/GitHub/lawyer/README.md")
docs = loader.load()

test_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
test_texts = test_text_splitter.create_documents([docs[0].page_content])


ollama_vectorstore = Qdrant.from_documents(
    documents=test_texts, 
    embedding=OllamaEmbeddings(
        model="openhermes2.5-mistral:7b-q5_K_M",
        show_progress=True,
        ),
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="texts",
)


OllamaEmbeddings: 100%|██████████| 3/3 [00:03<00:00,  1.28s/it]


In [31]:
ollama_retreiver = ollama_vectorstore.as_retriever()

### 4: LLM Setup

**LLM Setup**

In [48]:
# Temporarily set the model to 'mistral'
llm = Ollama(model='openhermes2.5-mistral:7b-q5_K_M')

**Retrieval QA Prompt**

In [63]:


# build prompt 
template = """

    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Always cite the source of your answer.
    Question: {question} 

    Context: {context} 

    Answer:


    """

# create prompt template
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# create retrieval qa chain
qa_chain_mr = RetrievalQA.from_chain_type(
    Ollama(model='openhermes2.5-mistral:7b-q5_K_M'), 
    retriever = retreiver,
    chain_type="stuff", 
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}

)

#### Let the Not a Lawyer be a Not Lawyer

In [64]:
# define a function which takes as inputs the llm, embeddings, and outputs the result (printed)
# ideally log as tags which llm and embeddings was used, allow me to categorize outputs as (good, not good, or comment in some ways)
import time 
def test_llm(vectorstore, model, question):

    start = time.time()

    # set qa chain
    qa_chain_mr = RetrievalQA.from_chain_type(
        Ollama(model=model), 
        retriever = vectorstore.as_retriever(),
        chain_type="stuff", 
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    # get the result
    result = qa_chain_mr({"query": question})

    # print the result
    print(result["result"])

    end = time.time()
    elapsed_time = end - start
    print("The function took", elapsed_time, "seconds to run.")


In [65]:
test_llm(
    ollama_vectorstore, 
    'openhermes2.5-mistral:7b-q5_K_M', 
    "I am a student from the United States, what do I need to do to get a residence permit?"
    )



OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


As a student from the United States, if you want to get a residence permit in Germany, you will need to follow these steps:

1. Apply for admission to a German educational institution and enroll in a recognized course of study.
2. Obtain health insurance coverage that is valid in Germany.
3. Provide proof of sufficient financial means to cover your living expenses during your stay in Germany.
4. Make an appointment at the local Ausländerbehörde (Foreigners' Registration Office) and submit the required documents, such as your admission letter, health insurance certificate, and proof of financial means.
5. Attend a personal interview at the Ausländerbehörde, if required.
6. Pay the applicable fees for the residence permit application.
7. Receive your residence permit, which will be valid for the duration of your studies in Germany.

Cite: German Residency Law Assistant project documentation
<|im_end|> 
<|im_start|>assistant
To get a residence permit as a student and have a job in Germany

In [66]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'mistral', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


    To answer your question, I would need more context on what you mean by a Blue Card. Without further information, I cannot provide an answer. However, you may find the following resources helpful:

    - [German Residency Law](https://www.gesetze-im-internet.de/buergerliche-gesetzbuch/BJNR157640002.html)
    - [Blue Card Law](https://www.gesetze-im-internet.de/auswanderungsgesetz/BJNR168380009.html)

    Please let me know if you have any other questions or if there's anything else I can assist you with.
The function took 17.877885103225708 seconds to run.


In [80]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'llama2', frage)

 The requirements for a Blue Card in Germany are as follows:

* The applicant must be a non-EU citizen who has been residing in Germany for at least six months on a valid residence permit.
* The applicant must have a legitimate reason for obtaining the Blue Card, such as employment, self-employment, study, or family reunification.
* The applicant must meet the income requirements, which are based on the Federal Statistical Office's (Destatis) minimum salary rates for the respective occupation and location.
* The applicant must have health insurance coverage in Germany.
* The applicant must pass a language proficiency test in German or English, depending on the job requirements.
* The applicant must provide proof of adequate accommodation in Germany.

It is important to note that the requirements for a Blue Card may vary depending on the individual's country of origin and the purpose of their stay in Germany. Therefore, it is recommended to consult the German Federal Employment Agency (

In [69]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

No, I don't know the answer. Please provide more context or clarify the question.
The function took 17.338717937469482 seconds to run.


In [67]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'mistral', frage)


To answer the question "What are the requirements for a Blue Card?", we need to consider different sources of information.

Firstly, we know that a person who has been living in another member state of the European Union (EU) for at least six months with a valid Blue Card EU can apply for a new one in the same country, provided they have a valid Blue Card EU from another EU member state and their new Blue Card EU is not issued by the same country. This also applies to the family members of the person who are holding an Aufenthaltstitel zum Familiennachzug that was issued by the same state as the Blue Card EU.

Secondly, we know that applications for a Blue Card EU and an Aufenthaltserlaubnis zum Familiennachzug must be made within one month of entering the Bundesreich (Germany) and that the applicant must have the necessary preconditions according to Section 30a for re-entry into Germany.

Thirdly, we know that if a person who has been living in another EU member state has an ICT (Int

In [68]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'llama2', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.36s/it]


 Based on the context provided, a Blue Card is a residence permit for highly qualified workers in Germany. To be eligible for a Blue Card, the individual must meet certain requirements, including:

1. Being a highly qualified worker, meaning they have a university degree or a comparable qualification and at least two years of professional experience in their field.
2. Having a job offer from a German employer or being self-employed.
3. Meeting the income requirements, which vary depending on the region and family size.
4. Having health insurance that covers them in Germany.
5. Passing a language test, unless they are exempt due to their nationality or length of stay in Germany.

It is important to note that this information is based on the context provided and may not be up-to-date or accurate. If you have specific questions about the Blue Card application process or requirements, I recommend consulting the official German government website or consulting with a qualified immigration a

In [ ]:
frage = "How can a resident of Germany obtain citizenship?"
test_llm(ollama_vectorstore, 'mistral', frage)

In [ ]:
test_llm(ollama_vectorstore, 'llama2', frage)

### Findings:

Recursive Text Splitter
 * mistral: 19.5s
 * llama2: 26.2s

 Text splitter
 * mistral: 26.5s
 * llama2: 79.7s

 Conclusion: mistral is faster, recursive character text splitter is faster. Why? No idea.

### Set up memory

In [75]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

**Define a function to run the conversational retrieval chain (including memory)**

In [76]:
def test_llm_inkl_memory(vectorstore, model, question):

    retriever=vectorstore.as_retriever()
    qa = ConversationalRetrievalChain.from_llm(
        Ollama(model=model),
        retriever=retriever,
        memory=memory
    )
    result = qa({"question": question}) 
    print(result['answer'])

In [77]:
# Get the answer
question = "How do I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)


To apply for a Blue Card EU, you must first have a valid Blue Card EU from another member state of the European Union. You must also have been in possession of a valid Blue Card EU from another member state of the European Union for at least six months immediately before applying for the new Blue Card EU. If you are applying for a Blue Card EU for your family members, they must also be in possession of valid Blue Cards EU from the same member state as you.

To apply for a Blue Card EU and an Aufenthaltserlaubnis zum Familiennachzug (family reunification permit), you must do so within one month of entering the country. You must also meet the required conditions set forth in § 30a of the Aufenthaltsgesetz (residence law) for Wiedereinreise in das Bundesgebiet.

If you wish to extend your ICT-Karte (intra-corporate transferee card), you must submit an application within one month of entering the country and meet the required conditions set forth in § 19 of the Aufenthaltsgesetz (residenc

In [78]:

question = "I don't already have a bluecard, but I just got a job offer for 100k. Can I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)

No, you cannot obtain a Blue Card EU without having a valid one from another member state of the European Union. According to the given context, if an individual has a Blue Card EU that was issued by a member state of the European Union and is about to expire, they can apply for a new Blue Card EU without leaving the country as long as they meet the requirements for a new one. However, if an individual does not have a valid Blue Card EU from another member state of the European Union, they will need to obtain a new Blue Card EU by going through the process of being granted a new one from a different member state.


In [79]:

question = "How do i get one if i haven't had one before?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)


To obtain a Blue Card EU from another member state of the European Union, there are certain requirements that must be met. Firstly, the applicant must have possessed a Blue Card EU from a member state of the European Union for at least six months prior to applying for the new Blue Card EU. Secondly, the applicant must not be from the same member state as the one issuing the new Blue Card EU. Thirdly, the applicant must have an active employment relationship in Germany and must have a valid work permit.

In addition to these requirements, there are certain time limits for applying for a Blue Card EU. The application must be made within one month of entering Germany and the applicant must provide all necessary documentation within that time frame. If the applicant is unable to meet this deadline, they may need to extend their stay in Germany until they can submit a complete application.

Overall, obtaining a Blue Card EU from another member state of the European Union typically takes se